In [ ]:
import pandas as pd
import numpy as np
import json
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from gensim.models import Word2Vec
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import random

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
prepared_dataset=pd.read_csv('/content/drive/MyDrive/Math23K_English_with_data_prep.csv')
#prepared_dataset.head()

In [ ]:
with open('/content/drive/MyDrive/equations_data.json') as f:
  prefix_equations_list = json.load(f)
#print(len(prefix_equations_list))


In [ ]:
prepared_questions=prepared_dataset['prepared_question'].values.tolist()
normal_questions=prepared_dataset['segmented_question'].values.tolist()
infix_equations=prepared_dataset['equation'].values.tolist()
#print(len(infix_equations))
answers=prepared_dataset['ans'].values.tolist()

In [ ]:
idx_to_remove=[i for i in range(len(prefix_equations_list)) if prefix_equations_list[i]=="no_prefix"]
prepared_questions=[prepared_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
normal_questions=[normal_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
infix_equations=[infix_equations[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
answers=[answers[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
prefix_equations_list=[equation for equation in prefix_equations_list if equation != "no_prefix"]


idx_to_remove_chinnese=[i for i in range(len(infix_equations)) if infix_equations[i]=="x=80千米/小时"]
print(idx_to_remove_chinnese)
if len(idx_to_remove_chinnese) >=1:
  prepared_questions.pop(idx_to_remove_chinnese[0])
  normal_questions.pop(idx_to_remove_chinnese[0])
  infix_equations.pop(idx_to_remove_chinnese[0])
  answers.pop(idx_to_remove_chinnese[0])
  prefix_equations_list.pop(idx_to_remove_chinnese[0])

# print(len(idx_to_remove))
# print(idx_to_remove)
# print(len(prefix_equations_list))
# print(len(prepared_questions))
# print(len(normal_questions)) 
# print(len(infix_equations))
# print(len(answers))

[10149]


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
temp_questions=list()
for elem in prepared_questions:
  temp_questions.append(word_tokenize(elem))
prepared_questions=temp_questions

In [ ]:
index_to_word_input=['<pad>','<sseq>','<eseq>']
for question in prepared_questions:
  for word in question:
    if(word not in index_to_word_input):
      index_to_word_input.append(word)
#print(len(index_to_word_input))
#print(index_to_word_input[0])
input_vocab_size=len(index_to_word_input)
input_vocab_size

7807

In [ ]:
word_to_index_input={'<pad>':0,'<sseq>':1,'<eseq>':2}
for question in prepared_questions:
  for word in question:
    if(word not in word_to_index_input.keys()):
      idx=index_to_word_input.index(word)
      word_to_index_input[word]=idx
#print(len(word_to_index_input))
#print(word_to_index_input["n0"])
#print(word_to_index_input["n1"])
#print(word_to_index_input["children"])
#print(word_to_index_input['<pad>'])

In [ ]:
index_to_word_output=['<pad>','<sseq>','<eseq>']
for equation in prefix_equations_list:
  for elem in equation:
    if elem not in index_to_word_output:
      index_to_word_output.append(elem)
# print(len(index_to_word_output))
# print(index_to_word_output[0])
output_vocab_size=len(index_to_word_output)
# print(output_vocab_size)
# print(index_to_word_output)

In [ ]:
word_to_index_output={'<pad>':0,'<sseq>':1,'<eseq>':2}
for word in index_to_word_output:
  if word not in word_to_index_output.keys():
    word_to_index_output[word]=index_to_word_output.index(word)
# print(len(word_to_index_output))
# print(word_to_index_output["n0"])

In [ ]:
prepared_questions_temp=list()
for question in prepared_questions:
  question_temp=[word_to_index_input['<sseq>']]
  for word in question:
    question_temp.append(word_to_index_input[word])
  question_temp.append(word_to_index_input['<eseq>'])
  prepared_questions_temp.append(question_temp)
# print(prepared_questions[11235])
prepared_questions=prepared_questions_temp
# print(prepared_questions[11235])

In [ ]:
print(prepared_questions[11235])

[1, 3, 15, 159, 44, 361, 22, 23, 47, 3, 48, 49, 30, 3, 15, 47, 3, 52, 49, 57, 35, 23, 54, 105, 190, 186, 47, 3, 48, 49, 18, 36, 37, 23, 25, 361, 59, 3, 75, 56, 41, 2]


In [ ]:
prefix_equations_temp=list()
for eq in prefix_equations_list:
  eq_temp=[word_to_index_output['<sseq>']]
  for word in eq:
    eq_temp.append(word_to_index_output[word])
  eq_temp.append(word_to_index_input['<eseq>'])
  prefix_equations_temp.append(eq_temp)
# print(prepared_questions[11235])
#prepared_questions=prepared_equations_temp
 #print(prepared_questions[11235])

In [ ]:
prefix_equations_temp[0]

[1, 3, 4, 5, 6, 7, 2]

In [ ]:
max_len=max(len(question) for question in prepared_questions)
#print(max_len)
prepared_questions= sequence.pad_sequences(prepared_questions, maxlen=max_len,padding="post")
# print(prepared_questions[2])
# print(len(prepared_questions))
#reviews.shape

In [ ]:
max_len_output=max(len(eq) for eq in prefix_equations_temp)
#print(max_len)
prefix_equations_temp= sequence.pad_sequences(prefix_equations_temp, maxlen=max_len_output,padding="post")
# print(prepared_questions[2])
# print(len(prepared_questions))
#reviews.shape

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader 
import torch.nn.functional as F
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
prepared_questions_train, prepared_questions_test, prefix_equations_list_train, prefix_equations_list_test = train_test_split(prepared_questions,prefix_equations_temp, train_size=0.9)
# print(len(prepared_questions_train))
# print(len(prefix_equations_list_train))
# print(len(prepared_questions_test))
# print(len(prefix_equations_list_test))

In [ ]:
prepared_questions_train, prepared_questions_val, prefix_equations_list_train, prefix_equations_list_val = train_test_split(prepared_questions_train,prefix_equations_list_train, train_size=0.9)

In [ ]:
 print(len(prepared_questions_train))
 print(len(prefix_equations_list_train))
 print(input_vocab_size , output_vocab_size)
 max_len
# print(len(prepared_questions_val))
# print(len(prefix_equations_list_val))

18212
18212
7807 32


137

In [ ]:
prefix_equations_list_train[0]

array([ 1,  4,  7,  3, 12,  9,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [ ]:
class EncoderLSTM(nn.Module):

  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = input_vocab_size
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = float(0.5)

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)


EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(7807, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    self.input_size = input_size

    # Output size of the word embedding NN
    self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(self.input_size, self.embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(self.hidden_size, self.output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = input_vocab_size
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = float(0.5)
output_size = output_vocab_size

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)


DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(7807, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=32, bias=True)
)


In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5,mode='train'):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]
    #print(source.shape , target.shape)
    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]ape

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state_encoder, cell_state_encoder = self.Encoder_LSTM(source)

  
    if mode=='test':
      x = torch.tensor([1]*batch_size).to(device) # Trigger token <SOS>
      #print(x)
      outputs = []
      check = torch.tensor([2]*batch_size).to(device)
      while ( torch.all(x.eq(check)) == False):
         
        output, hidden_state_decoder, cell_state_decoder = self.Decoder_LSTM(x, hidden_state_encoder, cell_state_encoder)
        outputs.append(output.cpu().detach().numpy())
        best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
        x = best_guess 
 
      return outputs

    
    target_len = target.shape[0]
    target_vocab_size = output_vocab_size
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    x = target[0] # Trigger token <SOS>
    #print(x)
    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state_decoder, cell_state_decoder = self.Decoder_LSTM(x, hidden_state_encoder, cell_state_encoder)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs
    '''
    return total_pred
    '''
model = Seq2Seq(encoder_lstm , decoder_lstm)
print(model)


Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7807, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7807, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=32, bias=True)
  )
)


In [ ]:
optimizer = torch.optim.Adam(model.parameters() , lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

In [ ]:
def get_accuracy(y_pred,target):
    
        y_pred = y_pred.permute(0,2,1)
        _, y_pred = torch.max(y_pred, dim = 2)
        total_acc =0
        for true,pred in zip(target,y_pred):
          tp=0
          total=0
          acc =0
          for t,t_h in zip(true,pred):
            if(t!=0):
              if(t==t_h):
                tp+=1
            total+=1
          acc = tp/total
          total_acc += acc

        
        acc = total_acc / target.shape[0]
        acc = acc * 100
        return acc

In [ ]:
model.to(device)

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7807, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7807, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=32, bias=True)
  )
)

In [ ]:
!pip install tqdm
from tqdm import trange
from tqdm import tqdm

In [ ]:
num_epochs = 100
best_loss = 999999
best_epoch = -1

batch_size = 128

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  epoch_loss = 0.0
  batch_idx = 0
  total_acc = 0
  model.train()

  
  #pbar = tqdm(total = len(prepared_questions_train)/batch_size)
  while (batch_idx+1)*batch_size < len(prepared_questions_train):

    start = batch_idx*batch_size

    input = torch.tensor(prepared_questions_train[start : start+batch_size]).to(device)
    target = torch.tensor(prefix_equations_list_train[start : start+batch_size]).to(device)

    # Pass the input and target for model's forward method
    output = model(input.transpose(0,1), target.transpose(0,1))

    output = output.permute(1,2,0)
   
    #print(output.shape)
    target = torch.LongTensor(target.cpu().detach().numpy()).to(device)
    #print(target.shape)    

    # Clear the accumulating gradients
    optimizer.zero_grad()

    with torch.no_grad():

      acc = get_accuracy(output,target)
      total_acc += acc

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()

    epoch_loss += loss.item()
    #writer.add_scalar("Training loss", loss, global_step=step)
    #print(loss.item(), acc)

    batch_idx +=1
    
    
  print("Epoch_Loss - {} , accuracy : {}".format(epoch_loss/batch_idx , total_acc/batch_idx))
  print()
  
  with torch.no_grad():

    model.eval()
    while (batch_idx+1)*batch_size < len(prepared_questions_test):

      start = batch_idx*batch_size

      input = torch.tensor(prepared_questions_test[start : start+batch_size]).to(device)
      target = torch.tensor(prefix_equations_list_test[start : start+batch_size]).to(device)

      # Pass the input and target for model's forward method
      output = model(input.transpose(0,1), target.transpose(0,1),mode='test')

      output = torch.tensor(output).permute(1,2,0)
   
      #print(output.shape)
      target = torch.LongTensor(target.cpu().detach().numpy()).to(device)
      #print(target.shape)    


    
      acc = get_accuracy(output,target)
      total_acc += acc

      batch_idx +=1

    print("Epoch accuracy : {}".format(total_acc/batch_idx))
    print()
  
print(epoch_loss / len(prefix_equations_list_train))

#score = bleu(test_data[1:100], model, german, english, device)
#print(f"Bleu score {score*100:.2f}")

Epoch - 1 / 100
Epoch_Loss - 2.162599009527287 , accuracy : 5.193917867671152

Epoch accuracy : 5.193917867671152

Epoch - 2 / 100
Epoch_Loss - 2.1322308543702246 , accuracy : 5.3565396740910645

Epoch accuracy : 5.3565396740910645

Epoch - 3 / 100
Epoch_Loss - 2.1359360453108667 , accuracy : 5.283609359646256

Epoch accuracy : 5.283609359646256

Epoch - 4 / 100
Epoch_Loss - 2.135104195332863 , accuracy : 5.266080494595487

Epoch accuracy : 5.266080494595487

Epoch - 5 / 100
Epoch_Loss - 2.1735972451492094 , accuracy : 5.217204389125458

Epoch accuracy : 5.217204389125458

Epoch - 6 / 100
Epoch_Loss - 2.1272969010850074 , accuracy : 5.320074516868657

Epoch accuracy : 5.320074516868657

Epoch - 7 / 100
Epoch_Loss - 2.1305168447360185 , accuracy : 5.2839932034064905

Epoch accuracy : 5.2839932034064905

Epoch - 8 / 100
Epoch_Loss - 2.124390931196616 , accuracy : 5.334660579757625

Epoch accuracy : 5.334660579757625

Epoch - 9 / 100
Epoch_Loss - 2.1312956088025805 , accuracy : 5.24343371

In [ ]:
  with torch.no_grad():

    input = torch.tensor(prepared_questions_train[0 : 20]).to(device)
    target = torch.tensor(prefix_equations_list_train[0 : 20]).to(device)

    # Pass the input and target for model's forward method
    output = model(input.transpose(0,1), target.transpose(0,1))

    output = output.permute(1,2,0)

    y_pred = output.permute(0,2,1)
    _, y_pred = torch.max(y_pred, dim = 2)
    total_acc =0
    for true,pred in zip(target,y_pred):
      tp=0
      total=0
      acc =0
      for t,t_h in zip(true,pred):
        if(t!=0):
          if(t==t_h):
            tp+=1
          total+=1
      acc = tp/total
      total_acc += acc

        
    acc = total_acc / target.shape[0]
    acc = acc * 100
    acc

In [ ]:
acc

30.37162837162837